In [1]:
import mne
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mne.io import read_raw_edf
from scipy.signal import welch

In [2]:
matplotlib.use('Qt5Agg')  

Channels marked as bad:
none
Channels marked as bad:
none


Import data  (subjects to exclude are: (S038, S088, S089, S092, S100,106 and S104))

In [8]:
#R04, R08, R12: Motor imagery (left/right fist)
#R06, R10, R14: Motor imagery (both fists/feet)

simple_raw ="C:\\Users\\t490\\Downloads\\EGG_Project\\files\\S001\\S001R14.edf"

In [9]:
raw=mne.io.read_raw_edf(simple_raw, preload=True)
raw

Extracting EDF parameters from C:\Users\t490\Downloads\EGG_Project\files\S001\S001R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...


<RawEDF | S001R14.edf, 64 x 20000 (125.0 s), ~9.8 MiB, data loaded>

In [10]:
raw.plot()

Using matplotlib as 2D backend.


<MNEBrowseFigure size 1920x974 with 4 Axes>

Events detection (T1=left hand ,T2=right hand)


In [11]:
events, event_id = mne.events_from_annotations(raw)
mne.viz.plot_events(events, event_id=event_id)

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]


<Figure size 640x480 with 1 Axes>

In [12]:
mne.datasets.eegbci.standardize(raw)
raw.set_montage('standard_1020', match_case=False)

<RawEDF | S001R14.edf, 64 x 20000 (125.0 s), ~9.8 MiB, data loaded>

Visualize sensors location

In [13]:
raw.plot_sensors(ch_type='eeg', show_names=True)


<Figure size 640x640 with 1 Axes>

BandPass Filtering

In [14]:
raw.filter(8., 30., fir_design='firwin', verbose=False)
print(f"Applying Notch Filter at 50 Hz")
raw.notch_filter(50, fir_design='firwin', verbose=False)

Applying Notch Filter at 50 Hz


<RawEDF | S001R14.edf, 64 x 20000 (125.0 s), ~9.8 MiB, data loaded>

In [15]:
raw.plot()  # apres filtrage

<MNEBrowseFigure size 1920x974 with 4 Axes>

Epoch Extraction and Labeling

In [16]:
events, event_id = mne.events_from_annotations(raw)
mi_event_id = {'both_fists': event_id['T1'], 'both_feet': event_id['T2']}
epochs = mne.Epochs(
    raw, 
    events, 
    event_id=mi_event_id, 
    tmin=0.5, 
    tmax=2.5, 
    baseline=(None, 0.5),
    preload=True
)

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
15 matching events found


Setting baseline interval to [0.5, 0.5] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 15 events and 321 original time points ...
0 bad epochs dropped


Artifact Rejection

In [17]:
reject_criteria = dict(eeg=100e-6)  
epochs.drop_bad(reject=reject_criteria)

    Rejecting  epoch based on EEG : ['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6', 'Fp1', 'Fpz', 'AF7', 'AF3', 'AFz', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'FT7', 'T7', 'T9', 'TP7', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz']
    Rejecting  epoch based on EEG : ['CP5', 'CPz', 'AFz', 'T7', 'T9', 'TP7', 'TP8', 'P7', 'P5', 'P1', 'Pz', 'P2', 'P4', 'P6', 'P8', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz']
    Rejecting  epoch based on EEG : ['FC5', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'Fp1', 'AF7', 'AF3', 'AFz', 'AF4', 'F7', 'F5', 'F3', 'F1', 'Fz', 'F2', 'F4', 'FT7', 'T7', 'T8', 'T9', 'TP7', 'P7', 'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'O1', 'Oz', 'O2', 'Iz']
    Rejecting  epoch based on EEG : ['FC5', 'FC3', 'FC1', 'FCz

C:\Users\t490\AppData\Local\Temp\ipykernel_13472\1434474443.py:2: RuntimeWarning: All epochs were dropped!
You might need to alter reject/flat-criteria or drop bad channels to avoid this. You can use Epochs.plot_drop_log() to see which channels are responsible for the dropping of epochs.
  epochs.drop_bad(reject=reject_criteria)


<Epochs | 0 events (all good), 0.5 – 2.5 s (baseline 0.5 – 0.5 s), ~76 KiB, data loaded,
 'both_fists': 0
 'both_feet': 0>

In [18]:
data = epochs.get_data()
labels = epochs.events[:, -1] - min(epochs.event_id.values())

print(f"\n--- Final Data Status ---")
print(f"Epochs (Trials, Channels, Timepoints): {data.shape}")
print(f"Labels (0=Both Fists, 1=Both Feet): {np.unique(labels, return_counts=True)}")


--- Final Data Status ---
Epochs (Trials, Channels, Timepoints): (0, 64, 321)
Labels (0=Both Fists, 1=Both Feet): (array([], dtype=int64), array([], dtype=int64))


C:\Users\t490\AppData\Local\Temp\ipykernel_13472\2445778548.py:1: RuntimeWarning: epochs._get_data() can't run because this Epochs-object is empty. You might want to check Epochs.drop_log or Epochs.plot_drop_log() to see why epochs were dropped.
  data = epochs.get_data()


In [20]:
import mne
import numpy as np
import matplotlib 
import matplotlib.pyplot as plt
from mne.io import read_raw_edf
from mne.datasets import eegbci
from scipy.signal import welch
matplotlib.use('Qt5Agg')  
# --- Configuration du Fichier et des Paramètres ---
subject_id = 1
run_id = 6 # Run 6: Imagery Left/Right Fist
tmin, tmax = 0.5, 2.5 
sfreq_target = 160 # Fréquence d'échantillonnage
low_pass, high_pass = 0.5, 50.0 # Bande de filtrage large (pour garder les fréquences nécessaires pour l'ICA)
rej_crit_mV = 250.0 # Critère de rejet d'artefacts (en microvolts)


simple_raw ="C:\\Users\\t490\\Downloads\\EGG_Project\\files\\S001\\S001R06.edf"
raw = read_raw_edf(simple_raw, preload=True, verbose=False)

# Normalisation des noms de canaux et sélection EEG
eegbci.standardize(raw)
raw.pick_types(eeg=True)
raw.set_montage('standard_1020', match_case=False)
print(f"Fréquence d'échantillonnage initiale: {raw.info['sfreq']} Hz")

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Fréquence d'échantillonnage initiale: 160.0 Hz


In [21]:
# --- 2.1 Ré-référencement (Spatial Filtering - CAR) ---
raw.set_eeg_reference('average', projection=True, verbose=False)

# --- 2.2 Filtrage Passe-Bande (0.5 - 50 Hz) ---
# Utilisation d'une large bande pour l'étape ICA
print(f"Filtrage Passe-Bande: {low_pass} Hz à {high_pass} Hz")
raw.filter(low_pass, high_pass, fir_design='firwin', verbose=False)
raw.notch_filter(50, fir_design='firwin', verbose=False) # Filtre notch 50 Hz

# --- 2.3 Extraction des Époques  ---
events, event_id = mne.events_from_annotations(raw, verbose=False)
# T1: Left Fist, T2: Right Fist
mi_event_id = {'left_hand': event_id['T1'], 'right_hand': event_id['T2']}

epochs = mne.Epochs(
    raw, events, event_id=mi_event_id, 
    tmin=tmin, tmax=tmax, 
    baseline=(None, 0.5), # Correction de baseline sur les premières 0.5s de l'époque
    preload=True, verbose=False
)
print(f"\nForme des Époques brutes: {epochs.get_data().shape}")

# Application de la Normalisation (Z-Score) pour le modèle DL
# Xnormalized = (X - μ) / σ
data_normalized = epochs.get_data()
data_normalized = (data_normalized - np.mean(data_normalized, axis=(0, 2), keepdims=True)) / np.std(data_normalized, axis=(0, 2), keepdims=True)

Filtrage Passe-Bande: 0.5 Hz à 50.0 Hz

Forme des Époques brutes: (15, 64, 321)


In [22]:
# --- 2.4 Suppression d'Artefacts (ICA) ---
ica = mne.preprocessing.ICA(n_components=30, random_state=99, max_iter='auto', verbose=False)
ica.fit(epochs.copy().filter(1., 35., verbose=False)) # Ajuster l'ICA sur une copie filtrée


epochs_clean = epochs.copy()
ica.apply(epochs_clean, verbose=False)

# --- 2.5 Rejet d'Époques Basé sur l'Amplitude ---
print(f"Nombre d'époques avant rejet: {len(epochs_clean)}")
reject_criteria = dict(eeg=rej_crit_mV * 1e-6) # 150 µV
epochs_clean.drop_bad(reject=reject_criteria, verbose=False)
print(f"Nombre d'époques après rejet: {len(epochs_clean)}")

data_final = epochs_clean.get_data()
print(f"Forme des Données Prétraitées: {data_final.shape}")

C:\Users\t490\AppData\Local\Temp\ipykernel_13472\3234415150.py:3: RuntimeWarning: filter_length (529) is longer than the signal (321), distortion is likely. Reduce filter length or filter a longer signal.
  ica.fit(epochs.copy().filter(1., 35., verbose=False)) # Ajuster l'ICA sur une copie filtrée


Fitting ICA to data using 64 channels (please be patient, this may take a while)
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 30 components


C:\Users\t490\AppData\Local\Temp\ipykernel_13472\3234415150.py:3: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs.copy().filter(1., 35., verbose=False)) # Ajuster l'ICA sur une copie filtrée


    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 5.1s.
Nombre d'époques avant rejet: 15
Nombre d'époques après rejet: 9
Forme des Données Prétraitées: (9, 64, 321)


C:\Users\t490\AppData\Local\Temp\ipykernel_13472\3234415150.py:7: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  ica.apply(epochs_clean, verbose=False)


In [23]:
# --- 3.1 Caractéristiques du Domaine Temporel ---
# Mean, Variance (Std-dev)
mean_features = np.mean(data_final, axis=2)
variance_features = np.var(data_final, axis=2)

print(f"\nCaractéristiques Temporelles (Mean) - Forme: {mean_features.shape}")

# Visualisation des signaux moyens par classe (C3 et C4)
evoked_L = epochs_clean['left_hand'].average()
evoked_R = epochs_clean['right_hand'].average()

# Affichage des Signaux Moyens Filtrés et Nettoyés
fig, ax = plt.subplots(1, 2, figsize=(14, 5))
mne.viz.plot_compare_evokeds(
    [evoked_L, evoked_R], 
    picks=['C3'], 
    title='Moyenne Temporelle (C3) - Gauche vs Droite', 
    colors=['blue', 'red'], 
    linestyles=['-', '--'], 
    axes=ax[0], 
    show=False,
    legend='upper right'
)
mne.viz.plot_compare_evokeds(
    [evoked_L, evoked_R], 
    picks=['C4'], 
    title='Moyenne Temporelle (C4) - Gauche vs Droite', 
    colors=['blue', 'red'], 
    linestyles=['-', '--'], 
    axes=ax[1], 
    show=False,
    legend='upper right'
)
ax[0].set_ylabel('Amplitude ($\mu$V)')
fig.tight_layout()
fig.show()

<>:34: SyntaxWarning: invalid escape sequence '\m'
<>:34: SyntaxWarning: invalid escape sequence '\m'
C:\Users\t490\AppData\Local\Temp\ipykernel_13472\4126741053.py:34: SyntaxWarning: invalid escape sequence '\m'
  ax[0].set_ylabel('Amplitude ($\mu$V)')



Caractéristiques Temporelles (Mean) - Forme: (9, 64)


C:\Users\t490\AppData\Local\Temp\ipykernel_13472\4126741053.py:35: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


In [24]:
# --- 3.2 Caractéristiques du Domaine Fréquentiel (PSD) ---

# Calcul de la PSD en utilisant la méthode de Welch pour chaque époque/canal
freqs, psd = welch(data_final * 1e6, fs=sfreq_target, nperseg=int(sfreq_target * 0.5), axis=-1)
psd_features = np.mean(psd[:, :, (freqs >= 8) & (freqs <= 30)], axis=2) # Moyenne de la puissance sur la bande 8-30 Hz

print(f"Caractéristiques PSD (Moyenne 8-30 Hz) - Forme: {psd_features.shape}")

# Visualisation de la PSD moyenne par classe (sur Cz)
epochs_L_psd = epochs_clean['left_hand'].copy().load_data()
epochs_R_psd = epochs_clean['right_hand'].copy().load_data()

fig, ax = plt.subplots(figsize=(8, 5))
epochs_L_psd.plot_psd(fmax=40, average=True, spatial_colors=False, axes=ax, color='blue', show=False, picks=['Cz'], dB=False)
epochs_R_psd.plot_psd(fmax=40, average=True, spatial_colors=False, axes=ax, color='red', show=False, picks=['Cz'], dB=False)

ax.legend(['Left Hand', 'Right Hand'])
ax.set_title('Densité Spectrale de Puissance (PSD) Moyenne (Cz)')
fig.tight_layout()
fig.show()

Caractéristiques PSD (Moyenne 8-30 Hz) - Forme: (9, 64)
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
    Using multitaper spectrum estimation with 7 DPSS windows
Plotting power spectral density (dB=False).
Averaging across epochs before plotting...
NOTE: plot_psd() is a legacy function. New code should use .compute_psd().plot().
    Using multitaper spectrum estimation with 7 DPSS windows
Plotting power spectral density (dB=False).
Averaging across epochs before plotting...


C:\Users\t490\AppData\Local\Temp\ipykernel_13472\3836854619.py:14: RuntimeWarning: The legacy plot_psd() method got an unexpected keyword argument 'axes', which is a parameter of Spectrum.plot(). Try rewriting as object.compute_psd(...).plot(..., axes=<whatever>).
  epochs_L_psd.plot_psd(fmax=40, average=True, spatial_colors=False, axes=ax, color='blue', show=False, picks=['Cz'], dB=False)
C:\Users\t490\AppData\Local\Temp\ipykernel_13472\3836854619.py:15: RuntimeWarning: The legacy plot_psd() method got an unexpected keyword argument 'axes', which is a parameter of Spectrum.plot(). Try rewriting as object.compute_psd(...).plot(..., axes=<whatever>).
  epochs_R_psd.plot_psd(fmax=40, average=True, spatial_colors=False, axes=ax, color='red', show=False, picks=['Cz'], dB=False)


In [25]:
# --- 3.3 Caractéristiques Temps-Fréquence (Wavelet) ---
from scipy.signal import butter, lfilter

def extract_wavelet_bands(data, sfreq):
    # Beta (16-32 Hz) et Mu (8-16 Hz)
    
    # Filtre Beta (16-32 Hz)
    nyq = 0.5 * sfreq
    low_beta = 16.0 / nyq
    high_beta = 32.0 / nyq
    b_beta, a_beta = butter(4, [low_beta, high_beta], btype='band')
    beta_band = lfilter(b_beta, a_beta, data, axis=-1)
    
    # Filtre Mu (8-16 Hz)
    low_mu = 8.0 / nyq
    high_mu = 16.0 / nyq
    b_mu, a_mu = butter(4, [low_mu, high_mu], btype='band')
    mu_band = lfilter(b_mu, a_mu, data, axis=-1)
    
    # Caractéristique: variance (puissance) des sous-bandes
    mu_variance = np.var(mu_band, axis=-1)
    beta_variance = np.var(beta_band, axis=-1)
    
    # Concaténation des caractéristiques Mu et Beta
    wavelet_features = np.concatenate([mu_variance, beta_variance], axis=1)
    return wavelet_features

wavelet_features = extract_wavelet_bands(data_final, sfreq_target)
print(f"\nCaractéristiques Wavelet (Mu/Beta Variance) - Forme: {wavelet_features.shape}")


Caractéristiques Wavelet (Mu/Beta Variance) - Forme: (9, 128)
